In [1]:
import json
import pandas as pd
import jieba
import vocab
import re
import numpy as np
import preprocess_data
import utils

In [21]:
file_path = 'data_gen/merge_300_test.csv'
df = pd.read_csv(file_path)

In [22]:
df = df[df['answer_start'] > -1]
df = df[df['answer_end'] > -1]

In [23]:
length = df['answer_end'] - df['answer_start'] + 1
df['answer_len'] = length

In [32]:
a = df['answer_len'].value_counts()

In [25]:
max(length)

129.0

In [33]:
a = a[list(range(100000))]
a = a[a.notnull()].cumsum() / len(df)
split_len = a[a > 0.99].index[0]

In [34]:
split_len

47